# chunker: default program

In [1]:
from chunker import *
import os

In [3]:
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

## Test

In [2]:
import gzip

In [11]:
def read_conll(handle, input_idx=0, label_idx=2):
    conll_data = []
    contents = re.sub(r'\n\s*\n', r'\n\n', handle.read())
    contents = contents.rstrip()
    for sent_string in contents.split('\n\n'):
        annotations = list(zip(*[word_string.split() for word_string in sent_string.split('\n') ]))
        assert(input_idx < len(annotations))
        if label_idx < 0:
            conll_data.append( annotations[input_idx] )
            logging.info("CoNLL: {}".format( " ".join(annotations[input_idx])))
        else:
            assert(label_idx < len(annotations))
            conll_data.append(( annotations[input_idx], annotations[label_idx] ))
            logging.info("CoNLL: {} ||| {}".format( " ".join(annotations[input_idx]), " ".join(annotations[label_idx])))
    return conll_data

In [18]:
with gzip.open(os.path.join('../data', 'train.txt.gz'), 'rt') as f:
    training_data = read_conll(f)
    # annotations

[('[UNK]', 'in', 'the', 'pound', 'is', 'widely', 'expected', 'to', 'take', 'another', 'sharp', 'dive', 'if', 'trade', 'figures', 'for', 'September', ',', 'due', 'for', 'release', 'tomorrow', ',', 'fail', 'to', 'show', 'a', 'substantial', 'improvement', 'from', 'July', 'and', 'August', "'s", 'near-record', '[UNK]', '.'), ('NN', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'IN', 'NNP', ',', 'JJ', 'IN', 'NN', 'NN', ',', 'VB', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'POS', 'JJ', 'NNS', '.'), ('B-NP', 'B-PP', 'B-NP', 'I-NP', 'B-VP', 'I-VP', 'I-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-SBAR', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'O', 'B-ADJP', 'B-PP', 'B-NP', 'B-NP', 'O', 'B-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'B-NP', 'I-NP', 'I-NP', 'O')]
[('Chancellor', 'of', 'the', 'Exchequer', 'Nigel', 'Lawson', "'s", 'restated', 'commitment', 'to', 'a', 'firm', 'monetary', 'policy', 'has', 'helped', 'to',

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
with gzip.open(os.path.join('../data', 'train.txt.gz'), 'rt') as f:
    training_data = read_conll(f)
    print(training_data)

(('[UNK]', 'in', 'the', 'pound', 'is', 'widely', 'expected', 'to', 'take', 'another', 'sharp', 'dive', 'if', 'trade', 'figures', 'for', 'September', ',', 'due', 'for', 'release', 'tomorrow', ',', 'fail', 'to', 'show', 'a', 'substantial', 'improvement', 'from', 'July', 'and', 'August', "'s", 'near-record', '[UNK]', '.'), ('B-NP', 'B-PP', 'B-NP', 'I-NP', 'B-VP', 'I-VP', 'I-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-SBAR', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'O', 'B-ADJP', 'B-PP', 'B-NP', 'B-NP', 'O', 'B-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'B-NP', 'I-NP', 'I-NP', 'O'))


In [3]:
chunker = LSTMTagger(os.path.join('../data', 'train.txt.gz'), os.path.join('../data', 'chunker'), '.tar')
print(chunker.word_to_ix)

{'[UNK]': 0, 'in': 1, 'the': 2, 'pound': 3, 'is': 4, 'widely': 5, 'expected': 6, 'to': 7, 'take': 8, 'another': 9, 'sharp': 10, 'dive': 11, 'if': 12, 'trade': 13, 'figures': 14, 'for': 15, 'September': 16, ',': 17, 'due': 18, 'release': 19, 'tomorrow': 20, 'fail': 21, 'show': 22, 'a': 23, 'substantial': 24, 'improvement': 25, 'from': 26, 'July': 27, 'and': 28, 'August': 29, "'s": 30, 'near-record': 31, '.': 32, 'Chancellor': 33, 'of': 34, 'Exchequer': 35, 'Nigel': 36, 'Lawson': 37, 'restated': 38, 'commitment': 39, 'firm': 40, 'monetary': 41, 'policy': 42, 'has': 43, 'helped': 44, 'prevent': 45, 'sterling': 46, 'over': 47, 'past': 48, 'week': 49, 'But': 50, 'analysts': 51, 'underlying': 52, 'support': 53, 'been': 54, 'eroded': 55, 'by': 56, 'chancellor': 57, 'failure': 58, 'announce': 59, 'any': 60, 'new': 61, 'measures': 62, 'his': 63, 'Mansion': 64, 'House': 65, 'speech': 66, 'last': 67, 'Thursday': 68, 'This': 69, 'increased': 70, 'risk': 71, 'government': 72, 'being': 73, 'forced':

In [2]:
chunker = LSTMTagger(os.path.join('../data', 'train.txt.gz'), os.path.join('../data', 'chunker'), '.tar')
print(chunker.training_data[0])

(('[UNK]', 'in', 'the', 'pound', 'is', 'widely', 'expected', 'to', 'take', 'another', 'sharp', 'dive', 'if', 'trade', 'figures', 'for', 'September', ',', 'due', 'for', 'release', 'tomorrow', ',', 'fail', 'to', 'show', 'a', 'substantial', 'improvement', 'from', 'July', 'and', 'August', "'s", 'near-record', '[UNK]', '.'), ('B-NP', 'B-PP', 'B-NP', 'I-NP', 'B-VP', 'I-VP', 'I-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-SBAR', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'O', 'B-ADJP', 'B-PP', 'B-NP', 'B-NP', 'O', 'B-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'B-NP', 'I-NP', 'I-NP', 'O'), ('NN', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'IN', 'NNP', ',', 'JJ', 'IN', 'NN', 'NN', ',', 'VB', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'POS', 'JJ', 'NNS', '.'))


In [3]:
aa = list(set(['NN', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'IN', 'NNP', ',', 'JJ', 'IN', 'NN', 'NN', ',', 'VB', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'POS', 'JJ', 'NNS', '.']))
print(aa)

['IN', 'DT', 'POS', 'VBN', 'NN', '.', 'TO', 'CC', 'VBZ', 'RB', 'VB', 'NNP', ',', 'JJ', 'NNS']


In [7]:
lst = []
lst.append((('[UNK]', 'in', 'the', 'pound', 'is', 'widely', 'expected', 'to', 'take', 'another', 'sharp', 'dive', 'if', 'trade', 'figures', 'for', 'September', ',', 'due', 'for', 'release', 'tomorrow', ',', 'fail', 'to', 'show', 'a', 'substantial', 'improvement', 'from', 'July', 'and', 'August', "'s", 'near-record', '[UNK]', '.'), ('B-NP', 'B-PP', 'B-NP', 'I-NP', 'B-VP', 'I-VP', 'I-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-SBAR', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'O', 'B-ADJP', 'B-PP', 'B-NP', 'B-NP', 'O', 'B-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'B-NP', 'I-NP', 'I-NP', 'O'), ('NN', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'IN', 'NNP', ',', 'JJ', 'IN', 'NN', 'NN', ',', 'VB', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'POS', 'JJ', 'NNS', '.')))
lst.append((('[UNK]', 'in', 'the', 'pound', 'is', 'widely', 'expected', 'to', 'take', 'another', 'sharp', 'dive', 'if', 'trade', 'figures', 'for', 'September', ',', 'due', 'for', 'release', 'tomorrow', ',', 'fail', 'to', 'show', 'a', 'substantial', 'improvement', 'from', 'July', 'and', 'August', "'s", 'near-record', '[UNK]', '.'), ('B-NP', 'B-PP', 'B-NP', 'I-NP', 'B-VP', 'I-VP', 'I-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-SBAR', 'B-NP', 'I-NP', 'B-PP', 'B-NP', 'O', 'B-ADJP', 'B-PP', 'B-NP', 'B-NP', 'O', 'B-VP', 'I-VP', 'I-VP', 'B-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'B-NP', 'I-NP', 'I-NP', 'O'), ('NN', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'IN', 'NNP', ',', 'JJ', 'IN', 'NN', 'NN', ',', 'VB', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'POS', 'JJ', 'NNS', '.')))

output = [tag for _, _, pos in lst for tag in pos]
print(output)

['NN', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'IN', 'NNP', ',', 'JJ', 'IN', 'NN', 'NN', ',', 'VB', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'POS', 'JJ', 'NNS', '.', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'IN', 'NNP', ',', 'JJ', 'IN', 'NN', 'NN', ',', 'VB', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'POS', 'JJ', 'NNS', '.']


In [5]:
# for index, item in enumerate(aa):
char_to_int = dict((c, i) for i, c in enumerate(aa))
print(char_to_int)

{'IN': 0, 'DT': 1, 'POS': 2, 'VBN': 3, 'NN': 4, '.': 5, 'TO': 6, 'CC': 7, 'VBZ': 8, 'RB': 9, 'VB': 10, 'NNP': 11, ',': 12, 'JJ': 13, 'NNS': 14}


## Extract tag data

In [28]:
import pandas as pd
import numpy as np
pos_lst = []
chunker = LSTMTagger(os.path.join('../data', 'train.txt.gz'), os.path.join('../data', 'chunker'), '.tar')
# s = pd.Series(list('abca'))
# torch.tensor(a, dtype=torch.long)
# print(chunker.training_data[0])
for sent, tags, pos_tag in chunker.training_data:
    pos_lst+= (list(pos_tag)[:-1])

s = pd.Series(pos_lst)
pos_dummies = pd.get_dummies(s)
pos_dummies.shape

(202791, 44)

In [29]:
pos_dummies_array = np.array(pos_dummies.values)
pos_dummies_array.shape

(202791, 44)

In [34]:
torch.tensor(pos_dummies_array, dtype=torch.long).view(pos_dummies_array.shape[1], 1, -1)

tensor([[[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 1, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]]])

In [14]:
s = (3,4,4,4,4,4,4,3)
print(list(s)[:-1])

[3, 4, 4, 4, 4, 4, 4]


## Run the default solution on dev

In [3]:
chunker = LSTMTagger(os.path.join('../data', 'train.txt.gz'), os.path.join('../data', 'chunker'), '.tar')
decoder_output = chunker.decode('../data/input/dev.txt')

100%|██████████| 1027/1027 [00:01<00:00, 936.27it/s]


## Evaluate the default output

In [5]:
flat_output = [ output for sent in decoder_output for output in sent ]
import conlleval
true_seqs = []
with open(os.path.join('data','reference','dev.out')) as r:
    for sent in conlleval.read_file(r):
        true_seqs += sent.split()
conlleval.evaluate(true_seqs, flat_output)

processed 23663 tokens with 11896 phrases; found: 11672 phrases; correct: 8568.
accuracy:  84.35%; (non-O)
accuracy:  85.65%; precision:  73.41%; recall:  72.02%; FB1:  72.71
             ADJP: precision:  36.49%; recall:  11.95%; FB1:  18.00  74
             ADVP: precision:  71.36%; recall:  39.45%; FB1:  50.81  220
            CONJP: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
             INTJ: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
               NP: precision:  70.33%; recall:  76.80%; FB1:  73.42  6811
               PP: precision:  92.40%; recall:  87.14%; FB1:  89.69  2302
              PRT: precision:  65.00%; recall:  57.78%; FB1:  61.18  40
             SBAR: precision:  84.62%; recall:  41.77%; FB1:  55.93  117
               VP: precision:  63.66%; recall:  58.25%; FB1:  60.83  2108


(73.40644276901988, 72.02420981842637, 72.70875763747455)

## Check torch.nn.embedding
- randomly generate words embedding (by running cells below several times)

In [5]:
word_to_ix = {"hello": 0, "pikachi": 1, "world": 2}
embeds = nn.Embedding(len(word_to_ix), 5)  # 2 words in vocab, 5 dimensional embeddings

lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)

lookup_tensor = torch.tensor([word_to_ix["world"]], dtype=torch.long)
world_embed = embeds(lookup_tensor)

lookup_tensor = torch.tensor([word_to_ix["pikachi"]], dtype=torch.long)
pikachi_embed = embeds(lookup_tensor)

print('hello embedding:', hello_embed)
print('world embedding:', world_embed)
print('pikachi embedding:', pikachi_embed)

cos = nn.CosineSimilarity(dim=1, eps=1e-6)
print(cos(hello_embed, world_embed))
print(cos(pikachi_embed, world_embed))
print(cos(pikachi_embed, hello_embed))

hello embedding: tensor([[ 0.2673, -0.4212, -0.5107, -1.5727, -0.1232]], grad_fn=<EmbeddingBackward>)
world embedding: tensor([[-0.4791,  1.3790,  2.5286,  0.4107, -0.9880]], grad_fn=<EmbeddingBackward>)
pikachi embedding: tensor([[ 3.5870, -1.8313,  1.5987, -1.2770,  0.3255]], grad_fn=<EmbeddingBackward>)
tensor([-0.4688], grad_fn=<DivBackward1>)
tensor([-0.0744], grad_fn=<DivBackward1>)
tensor([0.3675], grad_fn=<DivBackward1>)


## Documentation

Write some beautiful documentation of your program here.

## Analysis

Do some analysis of the results. What ideas did you try? What worked and what did not?